In [1]:
from deap import base, creator, tools, algorithms
import numpy as np
import random

INDS_LEN = 2000
NUM_LIMIT = 10
SUM_CONSTRAINT = 100
POP_SIZE = 50000
GEN_NUM = 60

step_size = 1 / (INDS_LEN / NUM_LIMIT)
block = [round(i * step_size, 3) for i in range(1, INDS_LEN // NUM_LIMIT+1)]
RESULT_LIST = block * NUM_LIMIT

print(block)

[0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.205, 0.21, 0.215, 0.22, 0.225, 0.23, 0.235, 0.24, 0.245, 0.25, 0.255, 0.26, 0.265, 0.27, 0.275, 0.28, 0.285, 0.29, 0.295, 0.3, 0.305, 0.31, 0.315, 0.32, 0.325, 0.33, 0.335, 0.34, 0.345, 0.35, 0.355, 0.36, 0.365, 0.37, 0.375, 0.38, 0.385, 0.39, 0.395, 0.4, 0.405, 0.41, 0.415, 0.42, 0.425, 0.43, 0.435, 0.44, 0.445, 0.45, 0.455, 0.46, 0.465, 0.47, 0.475, 0.48, 0.485, 0.49, 0.495, 0.5, 0.505, 0.51, 0.515, 0.52, 0.525, 0.53, 0.535, 0.54, 0.545, 0.55, 0.555, 0.56, 0.565, 0.57, 0.575, 0.58, 0.585, 0.59, 0.595, 0.6, 0.605, 0.61, 0.615, 0.62, 0.625, 0.63, 0.635, 0.64, 0.645, 0.65, 0.655, 0.66, 0.665, 0.67, 0.675, 0.68, 0.685, 0.69, 0.695, 0.7, 0.705, 0.71, 0.715, 0.72, 0.725, 0.73, 0.735, 0.74, 0.745, 0.75, 0.755, 0.76, 0.765, 0.77, 0.775

In [2]:
def evaluate(individual):
    if not feasible(individual):
        repair(individual)
    
    # r = 1
    # min_len = min(len(individual), len(RESULT_LIST))
    # for i in range(0, min_len):
    #     if individual[i] != 0:
    #         r *= individual[i] * RESULT_LIST[i]
    
    individual_array = np.array(individual)
    result_list_array = np.array(RESULT_LIST)
    min_len = min(len(individual), len(RESULT_LIST))
    individual_array = individual_array[:min_len]
    result_list_array = result_list_array[:min_len]
    product = individual_array * result_list_array
    product[individual_array == 0] = 1
    r = np.prod(product)
    
    return r, 

def feasible(inidividual):
    f1 = sum(inidividual) == SUM_CONSTRAINT
    
    i = 0
    for ind in inidividual:
        if ind != 0:
            i += 1
    f2 = i == NUM_LIMIT
    
    return f1 and f2

def repair(individual):
    non_zero_indices = [i for i, x in enumerate(individual) if x > 0]
    zero_indices = [i for i, x in enumerate(individual) if x == 0]

    if len(non_zero_indices) > NUM_LIMIT:
        selected_indices = random.sample(non_zero_indices, NUM_LIMIT)
    else:
        selected_indices = non_zero_indices + random.sample(zero_indices, NUM_LIMIT - len(non_zero_indices))
    
    for i in range(len(individual)):
        if i not in selected_indices:
            individual[i] = 0
    
    if sum(individual) == 0:
        new_individual = create_individual()
        individual[:] = new_individual
    else:
        n = SUM_CONSTRAINT / sum(individual)
        for i in range(len(individual)):
            individual[i] = int(individual[i] * n)

        d = SUM_CONSTRAINT - sum(individual)
        if selected_indices:
            chosen_index = random.choice(selected_indices)
            individual[chosen_index] += d
            
def create_individual():
    individual = [0] * INDS_LEN

    chosen_positions = random.sample(range(INDS_LEN), NUM_LIMIT)

    remaining = SUM_CONSTRAINT
    for i in range(NUM_LIMIT-1):
        individual[chosen_positions[i]] = random.randint(1, remaining - (NUM_LIMIT - (i + 1)))
        remaining -= individual[chosen_positions[i]]
    individual[chosen_positions[-1]] = remaining

    return individual

In [3]:
# Setup DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary Algorithm
def main():
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=GEN_NUM, stats=stats, halloffame=hof)

    return hof[0]

# Run the GA
best_ind = main()


gen	nevals	max                 
0  	50000 	(38683751.52089488,)
1  	29810 	(281451459.78159434,)
2  	30050 	(321028387.25715405,)
3  	30046 	(1253482570.1784487,)
4  	29963 	(1253482570.1784487,)
5  	29891 	(875827928.3738306,) 
6  	30244 	(1082720879.819334,) 
7  	29969 	(1708379055.8676975,)
8  	29865 	(1708379055.8676975,)
9  	29802 	(3096679434.7899694,)
10 	30172 	(3096679434.7899694,)
11 	30053 	(2680807195.090736,) 
12 	29965 	(3892063538.4573574,)
13 	29905 	(3511483992.8267446,)
14 	30219 	(4147880276.9653296,)
15 	30148 	(4596982260.994612,) 
16 	30003 	(4910982948.707082,) 
17 	30021 	(5143855254.543127,) 
18 	29946 	(5162995748.50821,)  
19 	30082 	(5162995748.50821,)  
20 	30092 	(5823099564.911807,) 
21 	29878 	(5919007141.477991,) 
22 	30160 	(6471056991.332739,) 
23 	30132 	(6471056991.332739,) 
24 	30019 	(6622290763.962261,) 
25 	30039 	(7600815721.548091,) 
26 	29886 	(7794201192.247225,) 
27 	30046 	(7794201192.247225,) 
28 	29958 	(7600815721.548091,) 
29 	29906 	(

In [4]:
for i in range(0, len(best_ind), int(len(best_ind)/NUM_LIMIT)):
    print(best_ind[i:i + int(len(best_ind)/NUM_LIMIT)])
    
lst = [x for x in best_ind if x !=0]
print("Best Individual:", lst, "Fitness:", best_ind.fitness.values[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0